In [1]:
import pandas as pd

Import all the files

In [2]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [3]:
print(visits.head())
print(cart.tail())
print(checkout.tail())
print(purchase.tail())

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                  user_id           cart_time
343  952be3a5-9240-484f-845c-a6b2cbed72ee 2017-05-02 05:40:00
344  59d2b685-383a-4fce-a325-35d985713a8b 2017-01-20 03:58:00
345  a1203970-92ae-4fc7-843e-e1740c1be9d5 2017-11-12 23:07:00
346  55de53ae-89f6-43ff-a26d-df284678aca1 2017-03-17 19:56:00
347  0ea4cc68-dae4-4e35-b3e0-f0889932e1b5 2017-05-12 08:53:00
                                  user_id       checkout_time
221  55de53ae-89f6-43ff-a26d-df284678aca1 2017-03-17 20:04:00
222  58545be1-0a55-4d26-b584-df26e258d0ed 2017-07-20 01:35:00
223  c62fdf2f-3b30-4624-8463-d3961e687997 2017-11-21 22:58:00
224  f3dd7631-3cf6-4

Step 2: Left merging visits and cart

In [4]:
visitsAndCart = pd.merge(visits, cart, how='left')

Step 3: How long is `visits_cart`?

In [5]:
visits = len(visitsAndCart)
print(visits)
print(visitsAndCart.tail())

2000
                                   user_id          visit_time cart_time
1995  33913ac2-03da-45ae-8fc3-fea39df827c6 2017-03-25 03:29:00       NaT
1996  4f850132-b99d-4623-80e6-6e61d003577e 2017-01-08 09:57:00       NaT
1997  f0830b9b-1f5c-4e74-b63d-3f847cc6ce70 2017-09-07 12:56:00       NaT
1998  b01bffa7-63ba-4cd3-9d93-eb1477c23831 2017-07-20 04:37:00       NaT
1999  0336ca81-8d68-443f-9248-ac0b8ad147d5 2017-11-15 10:11:00       NaT


Step 4: How many timestamps are null for `cart_time`?

In [6]:
nullCartCount = len(visitsAndCart[visitsAndCart.cart_time.isnull()])
nullCartCount

1652

Step 5: What percentage only visited?

In [7]:
visitNoCart = (nullCartCount / visits) * 100
visitNoCart

82.6

Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [8]:
lenCart = len(cart)
lenCheckout = len(checkout)
noCheckout = round(((lenCart - lenCheckout) / lenCart) * 100, 2)
noCheckout

35.06

Step 7: Merge it all together

In [9]:
# How do you want it merged? I guess I'll do a left merge so all the rows persist.
allData = visitsAndCart.merge(checkout, how='left').merge(purchase, how='left')
allData

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT
...,...,...,...,...,...
2103,33913ac2-03da-45ae-8fc3-fea39df827c6,2017-03-25 03:29:00,NaT,NaT,NaT
2104,4f850132-b99d-4623-80e6-6e61d003577e,2017-01-08 09:57:00,NaT,NaT,NaT
2105,f0830b9b-1f5c-4e74-b63d-3f847cc6ce70,2017-09-07 12:56:00,NaT,NaT,NaT
2106,b01bffa7-63ba-4cd3-9d93-eb1477c23831,2017-07-20 04:37:00,NaT,NaT,NaT


Step 8: % of users who got to checkout but did not purchase

In [10]:
checkedOut = allData[~allData.checkout_time.isnull()]
checkoutNoPurchase = allData[(~allData.checkout_time.isnull()) & (allData.purchase_time.isnull())]
checkoutNoPurchasePercent = round((len(checkoutNoPurchase) / len(checkedOut)) * 100, 2)
checkoutNoPurchasePercent

24.55

Step 9: check each part of the funnel, let's print all 3 of them again

In [11]:
print(str(visitNoCart) + "% of visitors did not add an item to their cart")
print(str(noCheckout) + "% of visitors that added an item to their cart did not go to checkout")
print(str(checkoutNoPurchasePercent) + "% of visitors that went to checkout did not complete the purchase")

82.6% of visitors did not add an item to their cart
35.06% of visitors that added an item to their cart did not go to checkout
24.55% of visitors that went to checkout did not complete the purchase


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [12]:
# Would be nice to have a suggestion of what to add... looking at the codecademy.com version of this project, they suggest
# calculating the average time from initial visit to final purchase
allData['visitTime'] = allData.purchase_time - allData.visit_time

Step 11: examine the results

In [13]:
print(allData.visitTime)

0                  NaT
1      0 days 00:44:00
2                  NaT
3                  NaT
4                  NaT
             ...      
2103               NaT
2104               NaT
2105               NaT
2106               NaT
2107               NaT
Name: visitTime, Length: 2108, dtype: timedelta64[ns]


Step 12: average time to purchase

In [14]:
print(allData.visitTime.mean())

0 days 00:43:12.380952380


In [15]:
# What is this line for??? The solution file also has it empty and the last step on the website is the one above...
# Thanks for looking!